In [1]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langgraph.checkpoint import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode


In [7]:
from langchain.llms import LlamaCpp #https://pypi.org/project/llama-cpp-python/
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# https://huggingface.co/bartowski/Meta-Llama-3-8B-Instruct-GGUF
# local_path="./Meta-Llama-3-8B-Instruct.Q2_K.gguf"
local_path="gguf_models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf"

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path=local_path,
    callback_manager=callback_manager,
    verbose=False
)

In [ ]:
# tools = [TavilySearchResults(max_results=1)]
# tool_node = ToolNode(tools)

In [8]:
model = llm

In [9]:
# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [10]:
# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
# workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
# workflow.add_conditional_edges(
#     # First, we define the start node. We use `agent`.
#     # This means these are the edges taken after the `agent` node is called.
#     "agent",
#     # Next, we pass in the function that will determine which node is called next.

# )


In [13]:
# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile()

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="日本語でクマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。自己紹介をしてください")]},
    config={"configurable": {"thread_id": 42}}
)

。
初めまして、私は作家の「Kaito」と申します。主に短編小説や詩を書いています。私の作品は、日常生活で感じるささいなことから、大きなテーマを探求するまで、幅広いジャンルをカバーしています。

以下は、私が書いた短編小説です。

タイトル: クマと海辺

プロット:

クマが海辺に来て、アザラシと友達になります。二人は一緒に浜で遊んだり、潮風に当たったりして過ごします。最終的には、クマは家に帰ることを決め、別れを惜しむアザラシに感謝の言葉を残して、海辺を後にするのでした。

私はこの短編小説を通じて、日常生活で感じるささいなことから、大きなテーマを探求するまで、幅広いジャンルをカバーしています。読者は私の作品を通じて、自分自身の内面や外部世界との関係性に気づくことができます。私はそのような気づきや新しい